In [1]:
import random 

In [2]:
def minHASH(arrays, num_perm):
  permutations = []
  count = 0
  while count < num_perm:
    perm = random.sample(range(0, len(arrays[0])), len(arrays[0]))
      # print(skm.mutual_info_score(arrays[0], perm))
    permutations.append(perm)
    count += 1
    
  signatures = []
  for i in range(len(arrays)):
    signature = []
    for permutation in permutations:
      for j in permutation:
        if(arrays[i][j] == 1):
          signature.append(j)
          break
    signatures.append(signature)
  return signatures
      